In [140]:
import pandas as pd  
     
df = pd.read_csv("IMDB Dataset.csv")  
   
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [141]:
from sklearn.model_selection import train_test_split
df = df.sample(frac=0.3, random_state=1)

X=df["review"]
y=df["sentiment"]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [142]:
sampleData

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [143]:
import re

def clean_text(text):
    """Remove HTML tags, URLs, mentions, hashtags, and newlines from a string."""
    # Remove HTML tags
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    text = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#,@]\S+)|[,.\n]", "", text)
    text = re.sub('\s+', ' ', text) 
    return text

X = [clean_text(item) for item in X]
print(X[0])

With No Dead Heroes you get stupid lines like that as this woefully abysmal action flick needs to be seen to be believed William Sanders is saved by his buddy Harry Cotter during an extraction in Vietnam but gets himself captured by the enemy Fast forward ten years and Harry is now a brainwashed Russian operative with a mind control microchip implanted in his brain His new Russian superior is Ivan played to the obscene hilt by Nick Nicholson who might I add not only doesn't attempt once to speak with a Russian accent but resembles more a gas station attendant in Kentucky with his stained teeth What is even more absurd is the fact that he was also the dialog coach for this film Soon William is re-recruited by the CIA to hunt Harry down He teams up with Barbara a freedom fighter who has infiltrated Ivan's El Salvador camp and soon the both of them are blowing up half of South America Some scenes are so jaw droppingly awful that it's a wonder why this film doesn't have more of a cult foll

In [144]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
print(X[0])
tokens=[]
maximumLength = max(len(word_tokenize(item)) for item in X)  # Find the maximum sequence length

for item in X:
    tokens += word_tokenize(item)
print(maximumLength)   

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/priyadarshni/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


With No Dead Heroes you get stupid lines like that as this woefully abysmal action flick needs to be seen to be believed William Sanders is saved by his buddy Harry Cotter during an extraction in Vietnam but gets himself captured by the enemy Fast forward ten years and Harry is now a brainwashed Russian operative with a mind control microchip implanted in his brain His new Russian superior is Ivan played to the obscene hilt by Nick Nicholson who might I add not only doesn't attempt once to speak with a Russian accent but resembles more a gas station attendant in Kentucky with his stained teeth What is even more absurd is the fact that he was also the dialog coach for this film Soon William is re-recruited by the CIA to hunt Harry down He teams up with Barbara a freedom fighter who has infiltrated Ivan's El Salvador camp and soon the both of them are blowing up half of South America Some scenes are so jaw droppingly awful that it's a wonder why this film doesn't have more of a cult foll

In [145]:
vocab = {word: idx + 1 for idx, word in enumerate(tokens)} 


In [146]:
print(y.value_counts())

sentiment
negative    7521
positive    7479
Name: count, dtype: int64


In [147]:
y = np.array([1 if label == 'positive' else 0 for label in y], dtype=np.int64)

In [149]:
import torch
tempX=[]
for idx,item in enumerate(X):
    encodedX=[]
    for word in word_tokenize(item):
        encodedX.append(vocab[word])
    while len(encodedX) < maximumLength:
        encodedX.append(0)

    tempX.append(encodedX)
    
        

In [150]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

tempX=np.array(tempX, dtype=np.int64)



In [151]:

print("Shape of tempX:", tempX.shape)
# print("Length of y:", len(y))

Shape of tempX: (15000, 1836)


In [152]:
X_train, X_test, y_train, y_test = train_test_split(tempX, y, test_size=0.2, random_state=42)


In [153]:
import torch

from torch.utils.data import DataLoader, TensorDataset
print(X[0])
train_dataset = TensorDataset(torch.Tensor(X_train),torch.Tensor(y_train))
test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))

With No Dead Heroes you get stupid lines like that as this woefully abysmal action flick needs to be seen to be believed William Sanders is saved by his buddy Harry Cotter during an extraction in Vietnam but gets himself captured by the enemy Fast forward ten years and Harry is now a brainwashed Russian operative with a mind control microchip implanted in his brain His new Russian superior is Ivan played to the obscene hilt by Nick Nicholson who might I add not only doesn't attempt once to speak with a Russian accent but resembles more a gas station attendant in Kentucky with his stained teeth What is even more absurd is the fact that he was also the dialog coach for this film Soon William is re-recruited by the CIA to hunt Harry down He teams up with Barbara a freedom fighter who has infiltrated Ivan's El Salvador camp and soon the both of them are blowing up half of South America Some scenes are so jaw droppingly awful that it's a wonder why this film doesn't have more of a cult foll

In [154]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [164]:

import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size)
        self.dropout=nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.dropout(out)
        out = self.fc(out)
        return out, hidden



In [165]:
model=RNN(maximumLength,5000,1)

In [166]:
model

RNN(
  (rnn): RNN(1836, 5000)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=5000, out_features=1, bias=True)
)

In [168]:
import torch.optim as optim

loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.001)


In [170]:
num_epochs = 20

for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    
    for idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        # Initialize hidden state
        hidden = None 
        outputs, hidden = model(inputs, hidden)
        
        loss = loss_fn(outputs.squeeze(), targets)  # Squeeze to match target shape
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        y_probability = torch.sigmoid(outputs)
        y_predicted = (y_probability > 0.5).float()
        accuracy = (y_predicted == targets).float().mean().item()  # Calculate mean accuracy
        
        # Accumulate epoch loss and accuracy
        epoch_loss += loss.item()
        epoch_accuracy += accuracy
        
    
    # Average epoch loss and accuracy
    epoch_loss /= len(train_loader)
    epoch_accuracy /= len(train_loader)
    
    # Print epoch summary
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

print('Training finished!')


Epoch [1/20], Loss: 3.2130, Accuracy: 0.5012
Epoch [2/20], Loss: 3.4080, Accuracy: 0.4992
Epoch [3/20], Loss: 3.3873, Accuracy: 0.5040
Epoch [4/20], Loss: 3.3764, Accuracy: 0.5065
Epoch [5/20], Loss: 3.6107, Accuracy: 0.4935
Epoch [6/20], Loss: 3.5538, Accuracy: 0.5000
Epoch [7/20], Loss: 3.3824, Accuracy: 0.5060
Epoch [8/20], Loss: 3.4575, Accuracy: 0.4990
Epoch [9/20], Loss: 3.4831, Accuracy: 0.4991
Epoch [10/20], Loss: 3.5068, Accuracy: 0.4973
Epoch [11/20], Loss: 3.4987, Accuracy: 0.5009
Epoch [12/20], Loss: 3.5623, Accuracy: 0.4978
Epoch [13/20], Loss: 3.3227, Accuracy: 0.5067
Epoch [14/20], Loss: 3.4532, Accuracy: 0.4962
Epoch [15/20], Loss: 3.3895, Accuracy: 0.5017
Epoch [16/20], Loss: 3.4633, Accuracy: 0.4974
Epoch [17/20], Loss: 3.3880, Accuracy: 0.5038
Epoch [18/20], Loss: 3.2216, Accuracy: 0.5046
Epoch [19/20], Loss: 3.4818, Accuracy: 0.4973
Epoch [20/20], Loss: 3.4523, Accuracy: 0.4951
Training finished!


In [171]:
correct = 0
total = 0
model.eval()

# Disable gradient calculation
with torch.no_grad():
    for idx, (inputs, targets) in enumerate(test_loader):
        hidden = None
        outputs, hidden = model(inputs, hidden)
        predicted = torch.round(outputs).squeeze()  # Assuming binary classification
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

# Calculate overall accuracy
accuracy = correct / total
print(f'Test Accuracy: {accuracy:.4f}')
        
   

Test Accuracy: 0.0103
